# Multi-Party Computation (MPC)
## 3-party computation
### Python implementation

@author: Ofer Rivlin<br>
@mail: ofer.rivlin@intel.com<br>

https://en.wikipedia.org/wiki/Secure_multi-party_computation

Multi-party computation (MPC) is an advanced topic in cryptography and its full implementation can be complex. However, a simple form of MPC can be explained using additive secret sharing over a finite field, such as the integers modulo some prime p.

In this example, let's consider a scenario where three parties (Alice, Bob, and Charlie) want to jointly compute the sum of their private inputs without revealing their inputs to each other.

In this example, each party first generates "shares" of their private input.<br />
These shares are random numbers such that their sum modulo p is equal to the input.<br />
The parties then exchange shares with each other.<br />
Each party then sums the shares they received and their own share to compute a partial sum.<br />
The total sum of their private inputs can be computed by summing these partial sums.

This is still a very simplified example and many practical aspects of MPC are not covered, such as error handling, security against malicious parties, and efficiency considerations.<br />
But it provides a basic understanding of how parties can jointly compute a function over their private inputs without revealing them.

In [1]:
import random

# We define a prime number for our modulo operations
p = 43

In [2]:
def generate_shares(input_value):
    """
    Generates random shares for an input value.
    """    
    shares = [input_value, 0, 0]
    for i in range(2):
      r = random.randint(0, p)
      shares[0] = shares[0] - r
      shares[i+1] = r %p
    shares[0] = shares[0] %p

    # sanity check
    assert sum(shares) %p == input_value %p
    return shares

In [3]:
class Alice:
  def __init__(self, input):
    self.input = input
    self.shares = generate_shares(input)

    # Alice sends one share to each of the other parties
    self.share_to_bob = self.shares[1]
    self.share_to_charlie = self.shares[2]
    self.bob = None
    self.charlie = None
    self.shared_addition_result = 0

  def register_parties(self, bob, charlie):
    self.bob = bob
    self.charlie = charlie

  def share_for_addition(self):
    return (self.shares[0] + 
            self.bob.share_to_alice + 
            self.charlie.share_to_alice) %p
  
  def shared_addition(self):
    self.shared_addition_result = (self.share_for_addition() + 
                                    self.bob.share_for_addition() +
                                    self.charlie.share_for_addition()) %p

In [4]:
class Bob:
  def __init__(self, input):
    self.input = input
    self.shares = generate_shares(input)

    # Bob only shares a one share with Alice
    self.share_to_alice = self.shares[0]
    
    self.alice = None
    self.charlie = None
    self.shared_addition_result = 0

  def register_parties(self, alice, charlie):
    self.alice = alice
    self.charlie = charlie

  def share_for_addition(self):
    return (self.shares[1] + 
            self.shares[2] + 
            self.alice.share_to_bob) %p
  
  def shared_addition(self):
    self.shared_addition_result = (self.share_for_addition() + 
                                   self.alice.share_for_addition() +
                                   self.charlie.share_for_addition()) %p

In [5]:
class Charlie:
  def __init__(self, input):
    self.input = input
    self.shares = generate_shares(input)

    # Charlie only shares a one share with Alice
    self.share_to_alice = self.shares[0]
    
    self.alice = None
    self.bob = None
    self.shared_addition_result = 0

  def register_parties(self, bob, alice):
    self.alice = alice
    self.bob = bob

  def share_for_addition(self):
    return (self.shares[1] + 
            self.shares[2] + 
            self.alice.share_to_charlie) %p
  
  def shared_addition(self):
    self.shared_addition_result = (self.share_for_addition() + 
                                   self.bob.share_for_addition() +
                                   self.alice.share_for_addition()) %p

In [6]:
alice_secret_input = 11
bob_secret_input = 12
charlie_secret_input = 13

# Each party generates shares for their private input
alice = Alice(input=alice_secret_input)
bob = Bob(input=bob_secret_input)
charlie = Charlie(input=charlie_secret_input)

expected_shared_sum = alice.input + bob.input + charlie.input

print("Alice shares: {}".format(alice.shares))
print("Bob shares: {}".format(bob.shares))
print("Charlie shares: {}".format(charlie.shares))

alice.register_parties(bob=bob, charlie=charlie)
bob.register_parties(alice=alice, charlie=charlie)
charlie.register_parties(bob=bob, alice=alice)

# print(alice.share_for_addition())
# print(bob.share_for_addition())
# print(charlie.share_for_addition())

alice.shared_addition()
bob.shared_addition()
charlie.shared_addition()

# Each one of the parties can now compute the sum of their private inputs
assert alice.shared_addition_result == \
       bob.shared_addition_result == \
       charlie.shared_addition_result == \
       expected_shared_sum

print("\nShared computation result: {}".format(alice.shared_addition_result))

print("\nShared addition successful!")

Alice shares: [29, 38, 30]
Bob shares: [38, 20, 40]
Charlie shares: [10, 33, 13]

Shared computation result: 36

Shared addition successful!


In [7]:
"""
Example of a possible run
"""

# Alice
#------
a = 11
a_r2 = 21
a_r3 = 36

a_s1 = (a - a_r2 - a_r3) % p
a_s2 = a_r2 % 43
a_s3 = a_r3 % 43

# print("alice: {},{},{}".format(a_s1,a_s2,a_s3))
assert (a_s1 + a_s2 + a_s3) %p == a %p

# Bob
#----
b = 12
b_r2 = 2
b_r3 = 17

b_s1 = (b - b_r2 - b_r3) % p
b_s2 = b_r2 % p
b_s3 = b_r3 % p

# print("bob: {},{},{}".format(b_s1,b_s2,b_s3))
assert (b_s1 + b_s2 + b_s3) %p == b %p

# Charlie
#--------
c = 13
c_r2 = 40
c_r3 = 31

c_s1 = (c - c_r2 - c_r3) % p
c_s2 = c_r2 % p
c_s3 = c_r3 % p


assert (c_s1 + c_s2 + c_s3) %p == c %p

# Addition
#=========

a_add = (a_s1 + b_s1 + c_s1) %p
b_add = (a_s2 + b_s2 + b_s3) %p
c_add = (a_s3 + c_s2 + c_s3) %p

# print("charlie: {},{},{}".format(c_s1,c_s2,c_s3))
assert (a_add + b_add + c_add) %p == (a + b + c)